In [9]:
import os
import torch
import pytorch_lightning as L
import matplotlib.pyplot as plt
from omegaconf import DictConfig

%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

# Get number of cores
print(os.cpu_count())

# Get number of threads
print(torch.get_num_threads())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
True
Tesla V100-SXM2-32GB
34072559616
72
36


In [ ]:
from src import rugd_preprocessing

rugd_preprocessing("/mnt/personal/kuceral4/datasets/RUGD_old", "/mnt/personal/kuceral4/datasets/RUGD")

In [7]:
from model import RoadModel
from pytorch_lightning.loggers import WandbLogger

from src import RoadDataModule, LogPredictionsCallback, val_checkpoint, regular_checkpoint


def main(cfg: DictConfig) -> None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RoadModel(cfg, device)
    datamodule = RoadDataModule(cfg)

    wandb_logger = WandbLogger(project="road-segmentation", name="baseline", log_model='all')

    trainer = L.Trainer(max_epochs=5, accelerator="gpu", devices=1, logger=wandb_logger, callbacks=[
        LogPredictionsCallback(),
        val_checkpoint,
        regular_checkpoint
    ])
    trainer.fit(model, datamodule=datamodule)
    # trainer.test(model, datamodule=datamodule)
    # trainer.predict(model, datamodule=datamodule)

In [8]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="conf"):
    config = compose(config_name="config")
    main(config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/appl/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


FileNotFoundError: [Errno 2] No such file or directory: 'data/RUGD/train.txt'